In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Pre-processing Stage

In [3]:
no = pd.read_csv('no_discount.csv')
all = pd.read_csv('all_discount.csv')
half = pd.read_csv('half_discount.csv')
prod = pd.read_excel('product.xlsx')

## Data Preparation

In [4]:
# Map user features to number
dataframes = {'all': all, 'half': half, 'no': no}
feature_cols = ['user_feature_1', 'user_feature_2', 'user_feature_3']
category_maps = {}

# 1. Build mapping for each feature across all dataframes
for col in feature_cols:
    # Collect unique values from all dataframes for this column
    unique_vals = pd.concat([df[col] for df in dataframes.values()]).unique()
    cat_map = {cat: i+1 for i, cat in enumerate(sorted(unique_vals))}
    category_maps[col] = cat_map

# 2. Apply the mapping to each dataframe
for df in dataframes.values():
    for col in feature_cols:
        df[col + '_num'] = df[col].map(category_maps[col])

# 3. Show mapping for each feature
for col in feature_cols:
    print(f"\nMapping for {col}:")
    for cat, idx in category_maps[col].items():
        print(f"{idx}: {cat}")


Mapping for user_feature_1:
1: Female
2: Male
3: Non-binary / third gender
4: Prefer not to say

Mapping for user_feature_2:
1: 1-2 years ago
2: 2-3 years ago
3: 3-4 yeas ago
4: Currently, I don't have a phone
5: Less than 1 year ago
6: More than 4 years ago

Mapping for user_feature_3:
1: Brand reputation
2: Design and appearance
3: Good price
4: Product quality


In [5]:
# Map choice to number (no discount)
# Define the mapping for 'choice'
choice_map = {
    "Do not purchase; save the $30 or use it for other expenses ($30 is roughly enough to cover two meals at a fast-food restaurant in the U.S.)": 0,
    "Color: Black; Style: Solid color; Weight: 2.66 pounds; Price: $24 (MSRP: $24)": 1,
    "Color: Black; Style: Gradient color; Weight: 2.66 pounds; Price: $26 (MSRP: $26)": 2,
    "Color: Dark blue; Style: Solid color; Weight: 2.61 pounds; Price: $25 (MSRP: $25)": 3,
    "Color: Dark blue; Style: Gradient color; Weight: 2.64 pounds; Price: $25 (MSRP: $25)": 4,
    "Color: Light blue; Style: Gradient color; Weight: 2.68 pounds; Price: $26 (MSRP: $26)": 5,
    "Color: White; Style: Solid color; Weight: 2.68 pounds; Price: $27 (MSRP: $27)": 6
}

# Apply the mapping
no['choice_num'] = no['choice'].map(choice_map)

In [6]:
# Map choice to number (all discount)
# Define the mapping for 'choice'
choice_map = {
    "Do not purchase; save the $30 or use it for other expenses ($30 is roughly enough to cover two meals at a fast-food restaurant in the U.S.)": 0,
    "Color: Black; Style: Solid color; Weight: 2.66 pounds; Price: $4.8 (MSRP: $24)": 1,
    "Color: Black; Style: Gradient color; Weight: 2.66 pounds; Price: $5.2 (MSRP: $26)": 2,
    "Color: Dark blue; Style: Solid color; Weight: 2.61 pounds; Price: $5 (MSRP: $25)": 3,
    "Color: Dark blue; Style: Gradient color; Weight: 2.64 pounds; Price: $5 (MSRP: $25)": 4,
    "Color: Light blue; Style: Gradient color; Weight: 2.68 pounds; Price: $5.2 (MSRP: $26)": 5,
    "Color: White; Style: Solid color; Weight: 2.68 pounds; Price: $5.4 (MSRP: $27)": 6
}

# Apply the mapping
all['choice_num'] = all['choice'].map(choice_map)

In [7]:
# Map choice to number (half discount)
# Define the mapping for 'choice'
choice_map = {
    "Do not purchase; save the $30 or use it for other expenses ($30 is roughly enough to cover two meals at a fast-food restaurant in the U.S.)": 0,
    "Color: Black; Style: Solid color; Weight: 2.66 pounds; Price: $4.8 (MSRP: $24)": 1,
    "Color: Black; Style: Gradient color; Weight: 2.66 pounds; Price: $26 (MSRP: $26)": 2,
    "Color: Dark blue; Style: Solid color; Weight: 2.61 pounds; Price: $25 (MSRP: $25)": 3,
    "Color: Dark blue; Style: Gradient color; Weight: 2.64 pounds; Price: $25 (MSRP: $25)": 4,
    "Color: Light blue; Style: Gradient color; Weight: 2.68 pounds; Price: $5.2 (MSRP: $26)": 5,
    "Color: White; Style: Solid color; Weight: 2.68 pounds; Price: $5.4 (MSRP: $27)": 6
}

# Apply the mapping
half['choice_num'] = half['choice'].map(choice_map)

In [8]:
# Normalization: user features
# Define the columns to scale
columns_to_scale = ['user_feature_1_num', 'user_feature_2_num', 'user_feature_3_num']

# Initialize the scaler
scaler = MinMaxScaler()

# Fit and transform the selected columns
half[columns_to_scale] = scaler.fit_transform(half[columns_to_scale])

In [9]:
# Normalization: product features and price
# Define the columns to scale
columns_to_scale = [
    'product_feature_1', 'product_feature_2', 'product_feature_3', 'price'
]

# Initialize the scaler
scaler = MinMaxScaler()

# Fit and transform the selected columns
prod[columns_to_scale] = scaler.fit_transform(prod[columns_to_scale])

In [10]:
# Set parameters
discount_percentage = 0.2
num_product = 6

In [11]:
# Check the number of valid data
num_row_no = len(no)
num_row_all = len(all)
num_row_half = len(half)
print(num_row_no)
print(num_row_all)
print(num_row_half)

291
278
266


# Analysis Stage

## True GTE

In [12]:
# All products are control (no discount)
# Merge the purchasing data with the product data
merged_no = no.merge(prod, left_on='choice_num', right_on='product_id', how='left')

# Calculate the total revenue
revenue_control = 0
revenue_control = merged_no['price'].sum() * 300/291

# All products are treated (all discount)
# Merge the purchasing data with the product data
merged_all = all.merge(prod, left_on='choice_num', right_on='product_id', how='left')

# Calculate the total revenue
revenue_treated = 0
revenue_treated = merged_all['price'].sum() * discount_percentage * 300/278

# Calculate the GTE
true = revenue_treated - revenue_control
print(true)

-68.6551789496238


## DIM Estimator

In [21]:
# Define the function to calculate the revenue
def calculate_total_revenue(df):
    revenue_treated = 0.0
    revenue_control = 0.0
    df['adjusted_price'] = df.apply(
        lambda row: row['price'] * discount_percentage if row['if_treated'] == 1 else row['price'], axis=1
    )
    for _, row in df.iterrows():
        if row['if_treated'] == 0:
            revenue_control += row['adjusted_price']
        elif row['if_treated'] == 1:
            revenue_treated += row['adjusted_price']
    return revenue_control, revenue_treated

# Merge the purchasing data with the product data
merged_half = half.merge(prod, left_on='choice_num', right_on='product_id', how='left')

# Calculate the revenue difference
revenue_control, revenue_treated = calculate_total_revenue(merged_half)

# Calculate the GTE
naive = (revenue_treated - revenue_control) * 600/266
naive
naive_pe = abs((naive - true)/true)
print(naive_pe)

0.11024849998145407


## MNL Estimator

In [14]:
# Product features (skip the first row)
feature_cols = ['product_feature_1', 'product_feature_2', 'product_feature_3']
X_product = torch.tensor(prod[feature_cols].iloc[1:].to_numpy(), dtype=torch.float)

# Price (skip first row)
price = torch.tensor(prod['price'].iloc[1:].to_numpy(), dtype=torch.float)

# Product randomization (skip first row)
prod_randomization = torch.tensor(prod['if_treated'].iloc[1:].to_numpy(), dtype=torch.bool)

# User features
user_num_cols = ['user_feature_1_num', 'user_feature_2_num', 'user_feature_3_num']
X_user = torch.tensor(merged_half[user_num_cols].to_numpy(),dtype=torch.float)

# User choice
choice = torch.tensor(
    merged_half['choice_num'].to_numpy(),
    dtype=torch.long
)

In [15]:
# Split the training and test set
X_user_train, X_user_test, choice_train, choice_test = train_test_split(
    X_user, choice, test_size=0.5, random_state=42
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X_user_train = X_user_train.to(device)
X_user_test  = X_user_test.to(device)
X_product    = X_product.to(device)
price        = price.to(device)
prod_randomization = prod_randomization.to(device)

choice_train = choice_train.to(device)
choice_test  = choice_test.to(device)

### No features

In [16]:
class LinearMNLModel_No(nn.Module):
    def __init__(self):
        super(LinearMNLModel_No, self).__init__()
        # v is the same constant value for all products.
        self.v = nn.Parameter(torch.tensor(1.0))
        # gamma is the price sensitivity parameter.
        self.gamma = nn.Parameter(torch.tensor(1.0))

    def forward(self, x_user, X_product, price, prod_randomization):
        N = x_user.shape[0]
        # Adjust prices for treated products (discounted)
        adjusted_price = torch.where(
            prod_randomization,
            price * discount_percentage,
            price
        )
        # Compute utilities: shape [M]
        utility = self.v + self.gamma * adjusted_price
        # Expand to shape [N, M]
        utility = utility.expand(N, -1)
        # Add outside option (utility = 0)
        zero_utilities = torch.zeros(N, 1, device=utility.device)
        utilities_with_outside = torch.cat((zero_utilities, utility), dim=1)
        return utilities_with_outside

In [17]:
# Model training
model = LinearMNLModel_No().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10000
l2_lambda = 1e-4  # Coefficient for L2 regularization
for epoch in range(num_epochs):
    optimizer.zero_grad()
    utilities = model(X_user_train, X_product, price, prod_randomization)
    utilities = utilities - utilities.max(dim=1, keepdim=True).values
    choice_probabilities = nn.functional.log_softmax(utilities, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train])

    # Compute L2 regularization penalty over all model parameters
    l2_norm = sum(param.pow(2.0).sum() for param in model.parameters())
    loss = loss + l2_lambda * l2_norm

    loss.backward()
    optimizer.step()
    if epoch % 1000 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

Epoch 0, Loss: 2.9594156742095947
Epoch 1000, Loss: 1.991666316986084
Epoch 2000, Loss: 1.457595944404602
Epoch 3000, Loss: 1.2737023830413818
Epoch 4000, Loss: 1.2315912246704102
Epoch 5000, Loss: 1.2191221714019775
Epoch 6000, Loss: 1.2120882272720337
Epoch 7000, Loss: 1.2091130018234253
Epoch 8000, Loss: 1.2084826231002808
Epoch 9000, Loss: 1.2084379196166992


In [20]:
# All control and all treated in test set
all_product_control = np.random.choice([0,1], num_product, p=[1, 0])
all_product_treated = np.random.choice([0,1], num_product, p=[0, 1])
all_product_control = torch.from_numpy(all_product_control).to(X_user_train.device).bool()
all_product_treated = torch.from_numpy(all_product_treated).to(X_user_train.device).bool()
X_user_test, X_product, price = X_user_test.to(device), X_product.to(device), price.to(device)

# Calculate expected revenue if all control
utilities = model(X_user_test, X_product, price, all_product_control)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum()
print(f"Expected Revenue: ${expected_revenue.item():.2f}")

# Calculate expected revenue if all treated
utilities = model(X_user_test, X_product, price, all_product_treated)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue_treated = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum() * discount_percentage
print(f"Expected Revenue: ${expected_revenue_treated.item():.2f}")

# Calculate GTE
linear = (expected_revenue_treated-expected_revenue).cpu().detach().numpy()
linear = linear * 600 / 266 # For comparison purpose
print(linear)
linear_no = abs((linear - true)/true)
print(linear_no)

Expected Revenue: $38.62
Expected Revenue: $8.52
-67.89952615149936
0.011006493751606264


### Only user features

In [22]:
class LinearMNLModel_UserOnly(nn.Module):
    def __init__(self, user_feature_dim):
        super(LinearMNLModel_UserOnly, self).__init__()
        self.beta_user = nn.Parameter(torch.randn(user_feature_dim))
        self.beta_price = nn.Parameter(torch.tensor(-1.0))

    def forward(self, x_user, X_product, price, prod_randomization):
        N, M = x_user.shape[0], X_product.shape[0]

        # Expand user features
        x_user_expanded = x_user.unsqueeze(1).expand(-1, M, -1).detach()
        utility_user = torch.sum(x_user_expanded * self.beta_user, dim=2)

        # Price adjustment
        adjusted_price = torch.where(
            prod_randomization.unsqueeze(0),
            price * discount_percentage,
            price
        )
        utility_price = adjusted_price * self.beta_price
        utility_price = utility_price.expand(N, M)

        total_utility = utility_user + utility_price

        # Outside option
        zero_utilities = torch.zeros(N, 1, device=total_utility.device)
        utilities_with_outside = torch.cat((zero_utilities,total_utility), dim=1)
        return utilities_with_outside

In [23]:
# Model training
model = LinearMNLModel_UserOnly(user_feature_dim=X_user.shape[1]).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10000
l2_lambda = 1e-4  # Coefficient for L2 regularization
for epoch in range(num_epochs):
    optimizer.zero_grad()
    utilities = model(X_user_train, X_product, price, prod_randomization)
    utilities = utilities - utilities.max(dim=1, keepdim=True).values
    choice_probabilities = nn.functional.log_softmax(utilities, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train])

    # Compute L2 regularization penalty over all model parameters
    l2_norm = sum(param.pow(2.0).sum() for param in model.parameters())
    loss = loss + l2_lambda * l2_norm

    loss.backward()
    optimizer.step()
    if epoch % 1000 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

Epoch 0, Loss: 2.138808488845825
Epoch 1000, Loss: 1.4613381624221802
Epoch 2000, Loss: 1.2889728546142578
Epoch 3000, Loss: 1.2356287240982056
Epoch 4000, Loss: 1.2093948125839233
Epoch 5000, Loss: 1.1977343559265137
Epoch 6000, Loss: 1.1932528018951416
Epoch 7000, Loss: 1.1921210289001465
Epoch 8000, Loss: 1.1920028924942017
Epoch 9000, Loss: 1.192000150680542


In [24]:
# All control and all treated in test set
all_product_control = np.random.choice([0,1], num_product, p=[1, 0])
all_product_treated = np.random.choice([0,1], num_product, p=[0, 1])
all_product_control = torch.from_numpy(all_product_control).to(X_user_train.device).bool()
all_product_treated = torch.from_numpy(all_product_treated).to(X_user_train.device).bool()
X_user_test, X_product, price = X_user_test.to(device), X_product.to(device), price.to(device)

# Calculate expected revenue if all control
utilities = model(X_user_test, X_product, price, all_product_control)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum()
print(f"Expected Revenue: ${expected_revenue.item():.2f}")

# Calculate expected revenue if all treated
utilities = model(X_user_test, X_product, price, all_product_treated)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue_treated = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum() * discount_percentage
print(f"Expected Revenue: ${expected_revenue_treated.item():.2f}")

# Calculate GTE
linear = (expected_revenue_treated-expected_revenue).cpu().detach().numpy()
linear = linear * 600 / 266
print(linear)
linear_user_only = abs((linear - true)/true)
print(linear_user_only)

Expected Revenue: $37.74
Expected Revenue: $8.91
-65.0199560294474
0.05294899781477188


### Only product features

In [25]:
class LinearMNLModel_ProductOnly(nn.Module):
    def __init__(self, product_feature_dim):
        super(LinearMNLModel_ProductOnly, self).__init__()
        self.beta_product = nn.Parameter(torch.randn(product_feature_dim))
        self.beta_price = nn.Parameter(torch.tensor(-1.0))

    def forward(self, x_user, X_product, price, prod_randomization):
        N, M = x_user.shape[0], X_product.shape[0]

        # Expand product features
        X_product_expanded = X_product.unsqueeze(0).expand(N, -1, -1).detach()
        utility_product = torch.sum(X_product_expanded * self.beta_product, dim=2)

        # Price adjustment
        adjusted_price = torch.where(
            prod_randomization.unsqueeze(0),
            price * discount_percentage,
            price
        )
        utility_price = adjusted_price * self.beta_price
        utility_price = utility_price.expand(N, M)

        total_utility = utility_product + utility_price

        # Outside option
        zero_utilities = torch.zeros(N, 1, device=total_utility.device)
        utilities_with_outside = torch.cat((zero_utilities,total_utility), dim=1)
        return utilities_with_outside

In [26]:
# Model training
model = LinearMNLModel_ProductOnly(product_feature_dim=X_product.shape[1]).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10000
l2_lambda = 1e-4  # Coefficient for L2 regularization
for epoch in range(num_epochs):
    optimizer.zero_grad()
    utilities = model(X_user_train, X_product, price, prod_randomization)
    utilities = utilities - utilities.max(dim=1, keepdim=True).values
    choice_probabilities = nn.functional.log_softmax(utilities, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train])

    # Compute L2 regularization penalty over all model parameters
    l2_norm = sum(param.pow(2.0).sum() for param in model.parameters())
    loss = loss + l2_lambda * l2_norm

    loss.backward()
    optimizer.step()
    if epoch % 1000 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

Epoch 0, Loss: 2.1990792751312256
Epoch 1000, Loss: 1.495612382888794
Epoch 2000, Loss: 1.278860092163086
Epoch 3000, Loss: 1.2116801738739014
Epoch 4000, Loss: 1.190086007118225
Epoch 5000, Loss: 1.184604287147522
Epoch 6000, Loss: 1.1838302612304688
Epoch 7000, Loss: 1.183793544769287
Epoch 8000, Loss: 1.183793306350708
Epoch 9000, Loss: 1.1837931871414185


In [27]:
# All control and all treated in test set
all_product_control = np.random.choice([0,1], num_product, p=[1, 0])
all_product_treated = np.random.choice([0,1], num_product, p=[0, 1])
all_product_control = torch.from_numpy(all_product_control).to(X_user_train.device).bool()
all_product_treated = torch.from_numpy(all_product_treated).to(X_user_train.device).bool()
X_user_test, X_product, price = X_user_test.to(device), X_product.to(device), price.to(device)

# Calculate expected revenue if all control
utilities = model(X_user_test, X_product, price, all_product_control)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum()
print(f"Expected Revenue: ${expected_revenue.item():.2f}")

# Calculate expected revenue if all treated
utilities = model(X_user_test, X_product, price, all_product_treated)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue_treated = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum() * discount_percentage
print(f"Expected Revenue: ${expected_revenue_treated.item():.2f}")

# Calculate GTE
linear = (expected_revenue_treated-expected_revenue).cpu().detach().numpy()
linear = linear * 600 / 266
print(linear)
linear_product_only = abs((linear - true)/true)
print(linear_product_only)

Expected Revenue: $25.70
Expected Revenue: $16.56
-20.61003061165487
0.6998037012360332


### All features

In [28]:
# MNL choice model
class LinearMNLModel(nn.Module):
    def __init__(self, user_feature_dim, product_feature_dim):
        super(LinearMNLModel, self).__init__()
        # Initialize parameters for user and product features
        self.beta_user = nn.Parameter(torch.randn(user_feature_dim))
        self.beta_product = nn.Parameter(torch.randn(product_feature_dim))
        self.beta_price = nn.Parameter(torch.tensor(-1.0))
    def forward(self, x_user, X_product, price, prod_randomization):
        N, M = x_user.shape[0], X_product.shape[0]

        # Expand user and product features to create a [N, M, F] shaped tensor for each
        x_user_expanded = x_user.unsqueeze(1).expand(-1, M, -1).detach()
        X_product_expanded = X_product.unsqueeze(0).expand(N, -1, -1).detach()

        # Calculate linear utility from features
        utility_user = torch.sum(x_user_expanded * self.beta_user, dim=2)
        utility_product = torch.sum(X_product_expanded * self.beta_product, dim=2)

        # Adjust prices based on randomization
        adjusted_price = torch.where(
            prod_randomization.unsqueeze(0),
            price * discount_percentage,
            price
        )

        # Calculate utility from price, properly expanding its dimension
        utility_price = adjusted_price * self.beta_price  # [M]
        utility_price = utility_price.expand(N, M)  # [N, M]

        # Total utility including features and price
        total_utility = utility_user + utility_product + utility_price

        # Incorporate the outside option with utility 0
        zero_utilities = torch.zeros(N, 1, device=total_utility.device)
        utilities_with_outside = torch.cat((zero_utilities,total_utility), dim=1)

        return utilities_with_outside

In [29]:
# Model training
model = LinearMNLModel(
    user_feature_dim=X_user.shape[1],
    product_feature_dim=X_product.shape[1]
).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10000
l2_lambda = 1e-4  # Coefficient for L2 regularization
for epoch in range(num_epochs):
    optimizer.zero_grad()
    utilities = model(X_user_train, X_product, price, prod_randomization)
    utilities = utilities - utilities.max(dim=1, keepdim=True).values
    choice_probabilities = nn.functional.log_softmax(utilities, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train])

    # Compute L2 regularization penalty over all model parameters
    l2_norm = sum(param.pow(2.0).sum() for param in model.parameters())
    loss = loss + l2_lambda * l2_norm

    loss.backward()
    optimizer.step()
    if epoch % 1000 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

Epoch 0, Loss: 2.50156831741333
Epoch 1000, Loss: 1.3553760051727295
Epoch 2000, Loss: 1.1897709369659424
Epoch 3000, Loss: 1.160108208656311
Epoch 4000, Loss: 1.1557260751724243
Epoch 5000, Loss: 1.1551049947738647
Epoch 6000, Loss: 1.154944658279419
Epoch 7000, Loss: 1.154920220375061
Epoch 8000, Loss: 1.1549186706542969
Epoch 9000, Loss: 1.1549185514450073


In [30]:
# All control and all treated in test set
all_product_control = np.random.choice([0,1], num_product, p=[1, 0])
all_product_treated = np.random.choice([0,1], num_product, p=[0, 1])
all_product_control = torch.from_numpy(all_product_control).to(X_user_train.device).bool()
all_product_treated = torch.from_numpy(all_product_treated).to(X_user_train.device).bool()
X_user_test, X_product, price = X_user_test.to(device), X_product.to(device), price.to(device)

# Calculate expected revenue if all control
utilities = model(X_user_test, X_product, price, all_product_control)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum()
print(f"Expected Revenue: ${expected_revenue.item():.2f}")

# Calculate expected revenue if all treated
utilities = model(X_user_test, X_product, price, all_product_treated)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue_treated = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum() * discount_percentage
print(f"Expected Revenue: ${expected_revenue_treated.item():.2f}")

# Calculate GTE
linear = (expected_revenue_treated-expected_revenue).cpu().detach().numpy()
linear = linear * 600 / 266
print(linear)
linear_all = abs((linear - true)/true)
print(linear_all)

Expected Revenue: $28.17
Expected Revenue: $12.08
-36.291435069607616
0.47139552143274294


## PDL Estimator

In [31]:
# Validation set
X_user_train1, X_user_val, choice_train1, choice_val = train_test_split(X_user_train, choice_train,test_size=0.1,random_state=34)

In [32]:
# Prepare data
def prepare_data(user_features, product_features, prices):
    num_products = product_features.shape[0]
    all_x_other_products = []
    all_other_prices = []
    for i in range(num_products):
        indices = [j for j in range(num_products) if j != i]
        other_products = product_features[indices].reshape(-1)
        other_product_prices = prices[indices]
        all_x_other_products.append(other_products)
        all_other_prices.append(other_product_prices)
    # Convert lists to tensor
    all_x_other_products = torch.stack(all_x_other_products, dim=0)
    all_other_prices = torch.stack(all_other_prices, dim=0)
    return user_features, product_features, prices, all_x_other_products, all_other_prices

### No features

In [33]:
class DeepMNLModel_No(nn.Module):
    """
    Price-only Deep MNL:
    u_ij = theta(p_j)
    Outside option utility = 0.
    """
    def __init__(self, num_product, hidden=5):
        super(DeepMNLModel_No, self).__init__()

        # theta(p_j):
        # Input:  [1]
        # Output: scalar utility
        self.theta = nn.Sequential(
            nn.Linear(1, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, X_product, X_other_products, x_other_prices, price, prod_randomization):
        """
        Inputs used:
        - price: [M]  own price p_j
        - prod_randomization: [M]  treatment indicator
        """
        device = price.device
        N, M = x_user.shape[0], price.shape[0]

        # Apply treatment discount to own prices
        adjusted_price = torch.where(
            prod_randomization.to(device).bool(),
            price * discount_percentage,
            price
        )  # [M]

        # Prepare input for theta: [M, 1]
        pj_input = adjusted_price.unsqueeze(1)

        # Expand to all users: [N, M, 1]
        theta_input_exp = pj_input.unsqueeze(0).expand(N, -1, -1)

        # u_ij = theta(p_j): [N, M]
        utilities = self.theta(theta_input_exp).squeeze(-1)

        # Outside option utility = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utilities], dim=1)

        return utilities_with_outside

In [34]:
# Apply the model
pdlmodel = DeepMNLModel_No(
    num_product=X_product.shape[0],
    hidden=5
).to(device)

# Weight initialization
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)

pdlmodel.apply(init_weights)

DeepMNLModel_No(
  (theta): Sequential(
    (0): Linear(in_features=1, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [35]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [36]:
# ----- Prepare validation data -----
prepared_val = prepare_data(
    X_user_val,
    X_product,
    price * (1 - (1 - discount_percentage) * prod_randomization)
)
user_features_val, product_features_val, _, all_x_other_products_val, all_other_prices_val = prepared_val

user_features_val = user_features_val.to(device)
product_features_val = product_features_val.to(device)
all_x_other_products_val = all_x_other_products_val.to(device)
all_other_prices_val = all_other_prices_val.to(device)
choice_val = choice_val.to(device)

# ----- Optimizer and scheduler -----
optimizer = optim.Adam(pdlmodel.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.5)
l2_lambda = 1e-4  # L2 regularization coefficient

# Early stopping setup
best_val_loss = float('inf')
patience = 15
patience_counter = 0

# ----- Training loop -----
for epoch in range(2000):
    pdlmodel.train()
    optimizer.zero_grad()

    # Forward pass (train): use prepared prices and other-prices matrix
    outputs = pdlmodel(
        user_features,         # x_user (not used inside DeepMNLModel_No, but kept for interface)
        product_features,      # X_product (not used)
        all_x_other_products,  # X_other_products (not used in No model)
        all_other_prices,      # x_other_prices: p_-j
        prices,                # price: experimental p_j (after discount)
        prod_randomization     # can be ignored inside No model
    )
    choice_probabilities = F.log_softmax(outputs, dim=1)

    # Training loss
    loss = -torch.mean(
        choice_probabilities[
            torch.arange(choice_probabilities.shape[0], device=device),
            choice_train1
        ]
    )

    # L2 regularization
    l2_norm = sum(param.pow(2.0).sum() for param in pdlmodel.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward
    loss.backward()
    optimizer.step()

    # ----- Validation -----
    pdlmodel.eval()
    with torch.no_grad():
        val_outputs = pdlmodel(
            user_features_val,
            product_features_val,
            all_x_other_products_val,
            all_other_prices_val,
            prices,              # same product prices for validation
            prod_randomization
        )
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(
            val_choice_probabilities[
                torch.arange(val_choice_probabilities.shape[0], device=device),
                choice_val
            ]
        )

    print(
        f"Epoch {epoch+1}, "
        f"Training Loss: {loss.item():.4f}, "
        f"Validation Loss: {val_loss.item():.4f}"
    )

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(pdlmodel.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

    scheduler.step()

Epoch 1, Training Loss: 1.9899, Validation Loss: 1.9602
Epoch 2, Training Loss: 1.9660, Validation Loss: 1.9387
Epoch 3, Training Loss: 1.9432, Validation Loss: 1.9212
Epoch 4, Training Loss: 1.9235, Validation Loss: 1.9042
Epoch 5, Training Loss: 1.9043, Validation Loss: 1.8872
Epoch 6, Training Loss: 1.8854, Validation Loss: 1.8704
Epoch 7, Training Loss: 1.8668, Validation Loss: 1.8534
Epoch 8, Training Loss: 1.8481, Validation Loss: 1.8362
Epoch 9, Training Loss: 1.8293, Validation Loss: 1.8187
Epoch 10, Training Loss: 1.8105, Validation Loss: 1.8011
Epoch 11, Training Loss: 1.7915, Validation Loss: 1.7832
Epoch 12, Training Loss: 1.7724, Validation Loss: 1.7652
Epoch 13, Training Loss: 1.7533, Validation Loss: 1.7471
Epoch 14, Training Loss: 1.7341, Validation Loss: 1.7294
Epoch 15, Training Loss: 1.7164, Validation Loss: 1.7121
Epoch 16, Training Loss: 1.6991, Validation Loss: 1.6939
Epoch 17, Training Loss: 1.6805, Validation Loss: 1.6748
Epoch 18, Training Loss: 1.6606, Validat

In [38]:
# All-control and all-treated for test set
num_product = X_product.shape[0]

# All control: product_randomization = 0
all_product_control = torch.zeros(num_product, device=device).bool()

# All treated: product_randomization = 1
all_product_treated = torch.ones(num_product, device=device).bool()

# Move base price and test users/products to device
X_user_test = X_user_test.to(device)
X_product = X_product.to(device)
price = price.to(device)

# --------- Expected revenue: all control ----------
# Control prices = base prices (no discount)
price_control = price.clone()

# Prepare test data under all-control prices
user_test_ctrl, product_test_ctrl, prices_ctrl, x_other_products_ctrl, other_prices_ctrl = prepare_data(
    X_user_test,
    X_product,
    price_control
)

utilities_ctrl = pdlmodel(
    user_test_ctrl,
    product_test_ctrl,
    x_other_products_ctrl,
    other_prices_ctrl,
    prices_ctrl,          # control prices
    all_product_control   # all False
)

probabilities_ctrl = F.softmax(utilities_ctrl, dim=1)

price_with_outside_ctrl = torch.cat(
    (torch.zeros(1, device=price.device), prices_ctrl), dim=0
)  # [M+1]

expected_revenue_ctrl = torch.sum(
    probabilities_ctrl * price_with_outside_ctrl.unsqueeze(0).expand_as(probabilities_ctrl),
    dim=1
).sum()

print(f"Expected Revenue (all control): ${expected_revenue_ctrl.item():.2f}")

# --------- Expected revenue: all treated ----------
# Treated prices = discounted prices
price_treated = price * discount_percentage

user_test_trt, product_test_trt, prices_trt, x_other_products_trt, other_prices_trt = prepare_data(
    X_user_test,
    X_product,
    price_treated
)

utilities_trt = pdlmodel(
    user_test_trt,
    product_test_trt,
    x_other_products_trt,
    other_prices_trt,
    prices_trt,          # treated prices
    all_product_treated  # all True
)

probabilities_trt = F.softmax(utilities_trt, dim=1)

price_with_outside_trt = torch.cat(
    (torch.zeros(1, device=price.device), prices_trt), dim=0
)

expected_revenue_trt = torch.sum(
    probabilities_trt * price_with_outside_trt.unsqueeze(0).expand_as(probabilities_trt),
    dim=1
).sum()

print(f"Expected Revenue (all treated): ${expected_revenue_trt.item():.2f}")

# --------- GTE ----------
pdl = (expected_revenue_trt - expected_revenue_ctrl).cpu().detach().numpy() * 600 / 266
print(pdl)

pdl_no = abs((pdl - true) / true)
print(pdl_no)


Expected Revenue (all control): $37.61
Expected Revenue (all treated): $8.84
-64.90133758774377
0.05467673989509868


### Only user features

In [49]:
class DeepMNLModel_UserOnly(nn.Module):
    """
    Deep MNL (user + own price only):
    u_ij = theta(x_i, p_j)
    Outside option utility = 0.
    """
    def __init__(self, user_dim, num_product, hidden=5):
        super(DeepMNLModel_UserOnly, self).__init__()

        # theta(x_i, p_j)
        # Input per (i,j): [user_dim + 1]
        self.theta = nn.Sequential(
            nn.Linear(user_dim + 1, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, X_product, X_other_products,
                x_other_prices, price, prod_randomization):
        """
        x_user: [N, F]   user features x_i
        price:  [M]      own price p_j (已经在外面处理好折扣的话，这里直接用)
        其它输入在该模型中不使用：
        - X_product, X_other_products, x_other_prices, prod_randomization
        """
        device = price.device
        N, M = x_user.shape[0], price.shape[0]

        # p_j: [M, 1]
        pj_feat = price.unsqueeze(1)          # [M, 1]

        # 扩展到所有用户： [N, M, 1]
        pj_feat_exp = pj_feat.unsqueeze(0).expand(N, -1, -1)

        # 用户特征扩展： [N, 1, F] -> [N, M, F]
        x_user_exp = x_user.unsqueeze(1).expand(-1, M, -1)

        # theta 输入: [N, M, F + 1]
        theta_input = torch.cat([x_user_exp, pj_feat_exp], dim=2)

        # reshape 成 [N*M, D]
        theta_input_flat = theta_input.reshape(N * M, -1)

        # u_ij: [N*M, 1] -> [N, M]
        utilities = self.theta(theta_input_flat).view(N, M)

        # outside option utility = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utilities], dim=1)

        return utilities_with_outside


In [50]:
# Apply the model
pdlmodel = DeepMNLModel_UserOnly(
    user_dim=X_user.shape[1],
    num_product=X_product.shape[0],
    hidden=5
).to(device)

# Weight initialization
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)

pdlmodel.apply(init_weights)

DeepMNLModel_UserOnly(
  (theta): Sequential(
    (0): Linear(in_features=4, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [51]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [52]:
# ----- Prepare validation data -----
prepared_val = prepare_data(
    X_user_val,
    X_product,
    price * (1 - (1 - discount_percentage) * prod_randomization)
)
user_features_val, product_features_val, _, all_x_other_products_val, all_other_prices_val = prepared_val

user_features_val = user_features_val.to(device)
product_features_val = product_features_val.to(device)
all_x_other_products_val = all_x_other_products_val.to(device)
all_other_prices_val = all_other_prices_val.to(device)
choice_val = choice_val.to(device)

# ----- Optimizer and scheduler -----
optimizer = optim.Adam(pdlmodel.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.5)
l2_lambda = 1e-4  # L2 regularization coefficient

# Early stopping setup
best_val_loss = float('inf')
patience = 15
patience_counter = 0

# ----- Training loop -----
for epoch in range(2000):
    pdlmodel.train()
    optimizer.zero_grad()

    # Forward pass (train): use prepared prices and other-prices matrix
    outputs = pdlmodel(
        user_features,         # x_user (not used inside DeepMNLModel_No, but kept for interface)
        product_features,      # X_product (not used)
        all_x_other_products,  # X_other_products (not used in No model)
        all_other_prices,      # x_other_prices: p_-j
        prices,                # price: experimental p_j (after discount)
        prod_randomization     # can be ignored inside No model
    )
    choice_probabilities = F.log_softmax(outputs, dim=1)

    # Training loss
    loss = -torch.mean(
        choice_probabilities[
            torch.arange(choice_probabilities.shape[0], device=device),
            choice_train1
        ]
    )

    # L2 regularization
    l2_norm = sum(param.pow(2.0).sum() for param in pdlmodel.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward
    loss.backward()
    optimizer.step()

    # ----- Validation -----
    pdlmodel.eval()
    with torch.no_grad():
        val_outputs = pdlmodel(
            user_features_val,
            product_features_val,
            all_x_other_products_val,
            all_other_prices_val,
            prices,              # same product prices for validation
            prod_randomization
        )
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(
            val_choice_probabilities[
                torch.arange(val_choice_probabilities.shape[0], device=device),
                choice_val
            ]
        )

    print(
        f"Epoch {epoch+1}, "
        f"Training Loss: {loss.item():.4f}, "
        f"Validation Loss: {val_loss.item():.4f}"
    )

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(pdlmodel.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

    scheduler.step()

Epoch 1, Training Loss: 1.9901, Validation Loss: 1.9655
Epoch 2, Training Loss: 1.9723, Validation Loss: 1.9547
Epoch 3, Training Loss: 1.9611, Validation Loss: 1.9457
Epoch 4, Training Loss: 1.9508, Validation Loss: 1.9377
Epoch 5, Training Loss: 1.9418, Validation Loss: 1.9301
Epoch 6, Training Loss: 1.9335, Validation Loss: 1.9224
Epoch 7, Training Loss: 1.9257, Validation Loss: 1.9152
Epoch 8, Training Loss: 1.9189, Validation Loss: 1.9083
Epoch 9, Training Loss: 1.9124, Validation Loss: 1.9016
Epoch 10, Training Loss: 1.9062, Validation Loss: 1.8953
Epoch 11, Training Loss: 1.9002, Validation Loss: 1.8895
Epoch 12, Training Loss: 1.8946, Validation Loss: 1.8839
Epoch 13, Training Loss: 1.8895, Validation Loss: 1.8784
Epoch 14, Training Loss: 1.8843, Validation Loss: 1.8729
Epoch 15, Training Loss: 1.8792, Validation Loss: 1.8674
Epoch 16, Training Loss: 1.8741, Validation Loss: 1.8619
Epoch 17, Training Loss: 1.8690, Validation Loss: 1.8564
Epoch 18, Training Loss: 1.8640, Validat

In [53]:
# All-control and all-treated for test set
num_product = X_product.shape[0]

# All control: product_randomization = 0
all_product_control = torch.zeros(num_product, device=device).bool()

# All treated: product_randomization = 1
all_product_treated = torch.ones(num_product, device=device).bool()

# Move base price and test users/products to device
X_user_test = X_user_test.to(device)
X_product = X_product.to(device)
price = price.to(device)

# --------- Expected revenue: all control ----------
# Control prices = base prices (no discount)
price_control = price.clone()

# Prepare test data under all-control prices
user_test_ctrl, product_test_ctrl, prices_ctrl, x_other_products_ctrl, other_prices_ctrl = prepare_data(
    X_user_test,
    X_product,
    price_control
)

utilities_ctrl = pdlmodel(
    user_test_ctrl,
    product_test_ctrl,
    x_other_products_ctrl,
    other_prices_ctrl,
    prices_ctrl,          # control prices
    all_product_control   # all False
)

probabilities_ctrl = F.softmax(utilities_ctrl, dim=1)

price_with_outside_ctrl = torch.cat(
    (torch.zeros(1, device=price.device), prices_ctrl), dim=0
)  # [M+1]

expected_revenue_ctrl = torch.sum(
    probabilities_ctrl * price_with_outside_ctrl.unsqueeze(0).expand_as(probabilities_ctrl),
    dim=1
).sum()

print(f"Expected Revenue (all control): ${expected_revenue_ctrl.item():.2f}")

# --------- Expected revenue: all treated ----------
# Treated prices = discounted prices
price_treated = price * discount_percentage

user_test_trt, product_test_trt, prices_trt, x_other_products_trt, other_prices_trt = prepare_data(
    X_user_test,
    X_product,
    price_treated
)

utilities_trt = pdlmodel(
    user_test_trt,
    product_test_trt,
    x_other_products_trt,
    other_prices_trt,
    prices_trt,          # treated prices
    all_product_treated  # all True
)

probabilities_trt = F.softmax(utilities_trt, dim=1)

price_with_outside_trt = torch.cat(
    (torch.zeros(1, device=price.device), prices_trt), dim=0
)

expected_revenue_trt = torch.sum(
    probabilities_trt * price_with_outside_trt.unsqueeze(0).expand_as(probabilities_trt),
    dim=1
).sum()

print(f"Expected Revenue (all treated): ${expected_revenue_trt.item():.2f}")

# --------- GTE ----------
pdl = (expected_revenue_trt - expected_revenue_ctrl).cpu().detach().numpy() * 600 / 266
print(pdl)

pdl_user = abs((pdl - true) / true)
print(pdl_user)

Expected Revenue (all control): $41.23
Expected Revenue (all treated): $8.25
-74.40124167535538
0.08369452696274797


### Only product features

In [110]:
class DeepMNLModel_ProductOnly(nn.Module):
    """
    Deep MNL (product-only):
    u_ij = theta(z_j, z_-j, p_j)
    Outside option utility = 0.
    """
    def __init__(self, product_dim, num_product, hidden=5):
        super(DeepMNLModel_ProductOnly, self).__init__()

        # Aggregate other products' features z_-j:
        # Input:  [M, (M-1)*F_p]
        # Output: [M, hidden]
        self.other_product_features_layers = nn.Sequential(
            nn.Linear(product_dim * (num_product - 1), hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU()
        )

        # theta(z_j, z_-j, p_j)
        # Input per (i,j): [F_p + hidden (z_-j) + 1 (p_j)]
        in_dim = product_dim + hidden + 1

        self.theta = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, X_product, X_other_products,
                x_other_prices, price, prod_randomization):
        """
        X_product:        [M, F_p]          product features z_j
        X_other_products: [M, (M-1)*F_p]    other products' features z_-j
        price:            [M]               own price p_j (already discounted)

        这里不再使用:
        - x_user, x_other_prices, prod_randomization
        """
        device = price.device
        # N users, M products
        N, M = x_user.shape[0], X_product.shape[0]

        # z_-j features: [M, hidden]
        zminus_feat = self.other_product_features_layers(X_other_products)

        # p_j scalar: [M, 1]
        pj_feat = price.unsqueeze(1)

        # Combine product-side info: [M, F_p + hidden + 1]
        product_feat = torch.cat(
            [X_product, zminus_feat, pj_feat],
            dim=1
        )

        # Expand to all users: [N, M, in_dim]
        product_feat_exp = product_feat.unsqueeze(0).expand(N, -1, -1)

        # Flatten to [N*M, in_dim]
        theta_input_flat = product_feat_exp.reshape(N * M, -1)

        # Utilities: [N*M, 1] -> [N, M]
        utilities = self.theta(theta_input_flat).view(N, M)

        # Outside option = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utilities], dim=1)

        return utilities_with_outside

In [111]:
# Apply the model
pdlmodel = DeepMNLModel_ProductOnly(
    product_dim=X_product.shape[1],
    num_product=X_product.shape[0],
    hidden=5
).to(device)

# Weight initialization
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)

pdlmodel.apply(init_weights)

DeepMNLModel_ProductOnly(
  (other_product_features_layers): Sequential(
    (0): Linear(in_features=15, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
  )
  (theta): Sequential(
    (0): Linear(in_features=9, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [112]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [113]:
# ----- Prepare validation data -----
prepared_val = prepare_data(
    X_user_val,
    X_product,
    price * (1 - (1 - discount_percentage) * prod_randomization)
)
user_features_val, product_features_val, _, all_x_other_products_val, all_other_prices_val = prepared_val

user_features_val = user_features_val.to(device)
product_features_val = product_features_val.to(device)
all_x_other_products_val = all_x_other_products_val.to(device)
all_other_prices_val = all_other_prices_val.to(device)
choice_val = choice_val.to(device)

# ----- Optimizer and scheduler -----
optimizer = optim.Adam(pdlmodel.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.5)
l2_lambda = 1e-4  # L2 regularization coefficient

# Early stopping setup
best_val_loss = float('inf')
patience = 15
patience_counter = 0

# ----- Training loop -----
for epoch in range(2000):
    pdlmodel.train()
    optimizer.zero_grad()

    # Forward pass (train): use prepared prices and other-prices matrix
    outputs = pdlmodel(
        user_features,         # x_user (not used inside DeepMNLModel_No, but kept for interface)
        product_features,      # X_product (not used)
        all_x_other_products,  # X_other_products (not used in No model)
        all_other_prices,      # x_other_prices: p_-j
        prices,                # price: experimental p_j (after discount)
        prod_randomization     # can be ignored inside No model
    )
    choice_probabilities = F.log_softmax(outputs, dim=1)

    # Training loss
    loss = -torch.mean(
        choice_probabilities[
            torch.arange(choice_probabilities.shape[0], device=device),
            choice_train1
        ]
    )

    # L2 regularization
    l2_norm = sum(param.pow(2.0).sum() for param in pdlmodel.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward
    loss.backward()
    optimizer.step()

    # ----- Validation -----
    pdlmodel.eval()
    with torch.no_grad():
        val_outputs = pdlmodel(
            user_features_val,
            product_features_val,
            all_x_other_products_val,
            all_other_prices_val,
            prices,              # same product prices for validation
            prod_randomization
        )
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(
            val_choice_probabilities[
                torch.arange(val_choice_probabilities.shape[0], device=device),
                choice_val
            ]
        )

    print(
        f"Epoch {epoch+1}, "
        f"Training Loss: {loss.item():.4f}, "
        f"Validation Loss: {val_loss.item():.4f}"
    )

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(pdlmodel.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

    scheduler.step()

Epoch 1, Training Loss: 1.8331, Validation Loss: 1.7368
Epoch 2, Training Loss: 1.7777, Validation Loss: 1.6712
Epoch 3, Training Loss: 1.7223, Validation Loss: 1.6078
Epoch 4, Training Loss: 1.6705, Validation Loss: 1.5450
Epoch 5, Training Loss: 1.6234, Validation Loss: 1.4814
Epoch 6, Training Loss: 1.5767, Validation Loss: 1.4198
Epoch 7, Training Loss: 1.5291, Validation Loss: 1.3595
Epoch 8, Training Loss: 1.4815, Validation Loss: 1.2979
Epoch 9, Training Loss: 1.4312, Validation Loss: 1.2412
Epoch 10, Training Loss: 1.3832, Validation Loss: 1.1907
Epoch 11, Training Loss: 1.3399, Validation Loss: 1.1477
Epoch 12, Training Loss: 1.3037, Validation Loss: 1.1123
Epoch 13, Training Loss: 1.2775, Validation Loss: 1.0923
Epoch 14, Training Loss: 1.2676, Validation Loss: 1.0870
Epoch 15, Training Loss: 1.2717, Validation Loss: 1.0923
Epoch 16, Training Loss: 1.2834, Validation Loss: 1.1005
Epoch 17, Training Loss: 1.2944, Validation Loss: 1.1057
Epoch 18, Training Loss: 1.2989, Validat

In [114]:
# All-control and all-treated for test set
num_product = X_product.shape[0]

# All control: product_randomization = 0
all_product_control = torch.zeros(num_product, device=device).bool()

# All treated: product_randomization = 1
all_product_treated = torch.ones(num_product, device=device).bool()

# Move base price and test users/products to device
X_user_test = X_user_test.to(device)
X_product = X_product.to(device)
price = price.to(device)

# --------- Expected revenue: all control ----------
# Control prices = base prices (no discount)
price_control = price.clone()

# Prepare test data under all-control prices
user_test_ctrl, product_test_ctrl, prices_ctrl, x_other_products_ctrl, other_prices_ctrl = prepare_data(
    X_user_test,
    X_product,
    price_control
)

utilities_ctrl = pdlmodel(
    user_test_ctrl,
    product_test_ctrl,
    x_other_products_ctrl,
    other_prices_ctrl,
    prices_ctrl,          # control prices
    all_product_control   # all False
)

probabilities_ctrl = F.softmax(utilities_ctrl, dim=1)

price_with_outside_ctrl = torch.cat(
    (torch.zeros(1, device=price.device), prices_ctrl), dim=0
)  # [M+1]

expected_revenue_ctrl = torch.sum(
    probabilities_ctrl * price_with_outside_ctrl.unsqueeze(0).expand_as(probabilities_ctrl),
    dim=1
).sum()

print(f"Expected Revenue (all control): ${expected_revenue_ctrl.item():.2f}")

# --------- Expected revenue: all treated ----------
# Treated prices = discounted prices
price_treated = price * discount_percentage

user_test_trt, product_test_trt, prices_trt, x_other_products_trt, other_prices_trt = prepare_data(
    X_user_test,
    X_product,
    price_treated
)

utilities_trt = pdlmodel(
    user_test_trt,
    product_test_trt,
    x_other_products_trt,
    other_prices_trt,
    prices_trt,          # treated prices
    all_product_treated  # all True
)

probabilities_trt = F.softmax(utilities_trt, dim=1)

price_with_outside_trt = torch.cat(
    (torch.zeros(1, device=price.device), prices_trt), dim=0
)

expected_revenue_trt = torch.sum(
    probabilities_trt * price_with_outside_trt.unsqueeze(0).expand_as(probabilities_trt),
    dim=1
).sum()

print(f"Expected Revenue (all treated): ${expected_revenue_trt.item():.2f}")

# --------- GTE ----------
pdl = (expected_revenue_trt - expected_revenue_ctrl).cpu().detach().numpy() * 600 / 266
print(pdl)

pdl_product = abs((pdl - true) / true)
print(pdl_product)

Expected Revenue (all control): $43.73
Expected Revenue (all treated): $11.41
-72.88818646194343
0.06165605533452369


### All features

In [120]:
# PDL choice model
class DeepMNLModel_UserProduct(nn.Module):
    """
    Deep MNL:
    u_ij = theta(x_i, z_j, z_-j, p_j)
    Outside option utility = 0.
    """
    def __init__(self, user_dim, product_dim, num_product, hidden=5):
        super(DeepMNLModel_UserProduct, self).__init__()

        # Aggregate other products' features z_-j:
        # Input:  [M, (M-1)*F_p]
        # Output: [M, hidden]
        self.other_product_features_layers = nn.Sequential(
            nn.Linear(product_dim * (num_product - 1), hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU()
        )

        # theta(x_i, z_j, z_-j, p_j)
        # Input per (i,j):
        #   user_dim (x_i)
        # + product_dim (z_j)
        # + hidden (z_-j)
        # + 1 (p_j)
        in_dim = user_dim + product_dim + hidden + 1

        self.theta = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, X_product, X_other_products,
                x_other_prices, price, prod_randomization):
        """
        x_user:          [N, F_u]          user features x_i
        X_product:       [M, F_p]          product features z_j
        X_other_products:[M, (M-1)*F_p]    other products' features z_-j
        price:           [M]               own price p_j (already discounted)

        x_other_prices, prod_randomization are not used here.
        """
        device = price.device
        N, M = x_user.shape[0], X_product.shape[0]

        # z_-j features: [M, hidden]
        zminus_feat = self.other_product_features_layers(X_other_products)

        # p_j: [M, 1]
        pj_feat = price.unsqueeze(1)

        # Product-side block: [M, F_p + hidden + 1]
        product_block = torch.cat(
            [X_product, zminus_feat, pj_feat],
            dim=1
        )

        # Expand product block for all users: [N, M, *]
        product_block_exp = product_block.unsqueeze(0).expand(N, -1, -1)

        # Expand user features: [N, 1, F_u] -> [N, M, F_u]
        x_user_exp = x_user.unsqueeze(1).expand(-1, M, -1)

        # Final theta input: [N, M, in_dim]
        theta_input = torch.cat([x_user_exp, product_block_exp], dim=2)

        # Flatten to [N*M, in_dim]
        theta_input_flat = theta_input.reshape(N * M, -1)

        # Utilities: [N*M, 1] -> [N, M]
        utilities = self.theta(theta_input_flat).view(N, M)

        # Outside option = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utilities], dim=1)

        return utilities_with_outside

In [121]:
# Apply the model
pdlmodel = DeepMNLModel_UserProduct(
    user_dim=X_user.shape[1],
    product_dim=X_product.shape[1],
    num_product=X_product.shape[0],
    hidden=5
).to(device)

# Weight initialization
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)

pdlmodel.apply(init_weights)

DeepMNLModel_UserProduct(
  (other_product_features_layers): Sequential(
    (0): Linear(in_features=15, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
  )
  (theta): Sequential(
    (0): Linear(in_features=12, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [122]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [123]:
# ----- Prepare validation data -----
prepared_val = prepare_data(
    X_user_val,
    X_product,
    price * (1 - (1 - discount_percentage) * prod_randomization)
)
user_features_val, product_features_val, _, all_x_other_products_val, all_other_prices_val = prepared_val

user_features_val = user_features_val.to(device)
product_features_val = product_features_val.to(device)
all_x_other_products_val = all_x_other_products_val.to(device)
all_other_prices_val = all_other_prices_val.to(device)
choice_val = choice_val.to(device)

# ----- Optimizer and scheduler -----
optimizer = optim.Adam(pdlmodel.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.5)
l2_lambda = 1e-4  # L2 regularization coefficient

# Early stopping setup
best_val_loss = float('inf')
patience = 15
patience_counter = 0

# ----- Training loop -----
for epoch in range(2000):
    pdlmodel.train()
    optimizer.zero_grad()

    # Forward pass (train): use prepared prices and other-prices matrix
    outputs = pdlmodel(
        user_features,         # x_user (not used inside DeepMNLModel_No, but kept for interface)
        product_features,      # X_product (not used)
        all_x_other_products,  # X_other_products (not used in No model)
        all_other_prices,      # x_other_prices: p_-j
        prices,                # price: experimental p_j (after discount)
        prod_randomization     # can be ignored inside No model
    )
    choice_probabilities = F.log_softmax(outputs, dim=1)

    # Training loss
    loss = -torch.mean(
        choice_probabilities[
            torch.arange(choice_probabilities.shape[0], device=device),
            choice_train1
        ]
    )

    # L2 regularization
    l2_norm = sum(param.pow(2.0).sum() for param in pdlmodel.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward
    loss.backward()
    optimizer.step()

    # ----- Validation -----
    pdlmodel.eval()
    with torch.no_grad():
        val_outputs = pdlmodel(
            user_features_val,
            product_features_val,
            all_x_other_products_val,
            all_other_prices_val,
            prices,              # same product prices for validation
            prod_randomization
        )
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(
            val_choice_probabilities[
                torch.arange(val_choice_probabilities.shape[0], device=device),
                choice_val
            ]
        )

    print(
        f"Epoch {epoch+1}, "
        f"Training Loss: {loss.item():.4f}, "
        f"Validation Loss: {val_loss.item():.4f}"
    )

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(pdlmodel.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

    scheduler.step()

Epoch 1, Training Loss: 2.2074, Validation Loss: 2.2007
Epoch 2, Training Loss: 2.1324, Validation Loss: 2.1293
Epoch 3, Training Loss: 2.0762, Validation Loss: 2.0685
Epoch 4, Training Loss: 2.0300, Validation Loss: 2.0168
Epoch 5, Training Loss: 1.9931, Validation Loss: 1.9748
Epoch 6, Training Loss: 1.9640, Validation Loss: 1.9424
Epoch 7, Training Loss: 1.9404, Validation Loss: 1.9167
Epoch 8, Training Loss: 1.9201, Validation Loss: 1.8926
Epoch 9, Training Loss: 1.9011, Validation Loss: 1.8735
Epoch 10, Training Loss: 1.8836, Validation Loss: 1.8556
Epoch 11, Training Loss: 1.8661, Validation Loss: 1.8358
Epoch 12, Training Loss: 1.8481, Validation Loss: 1.8156
Epoch 13, Training Loss: 1.8305, Validation Loss: 1.7962
Epoch 14, Training Loss: 1.8132, Validation Loss: 1.7768
Epoch 15, Training Loss: 1.7960, Validation Loss: 1.7573
Epoch 16, Training Loss: 1.7791, Validation Loss: 1.7378
Epoch 17, Training Loss: 1.7622, Validation Loss: 1.7183
Epoch 18, Training Loss: 1.7451, Validat

In [124]:
# All-control and all-treated for test set
num_product = X_product.shape[0]

# All control: product_randomization = 0
all_product_control = torch.zeros(num_product, device=device).bool()

# All treated: product_randomization = 1
all_product_treated = torch.ones(num_product, device=device).bool()

# Move base price and test users/products to device
X_user_test = X_user_test.to(device)
X_product = X_product.to(device)
price = price.to(device)

# --------- Expected revenue: all control ----------
# Control prices = base prices (no discount)
price_control = price.clone()

# Prepare test data under all-control prices
user_test_ctrl, product_test_ctrl, prices_ctrl, x_other_products_ctrl, other_prices_ctrl = prepare_data(
    X_user_test,
    X_product,
    price_control
)

utilities_ctrl = pdlmodel(
    user_test_ctrl,
    product_test_ctrl,
    x_other_products_ctrl,
    other_prices_ctrl,
    prices_ctrl,          # control prices
    all_product_control   # all False
)

probabilities_ctrl = F.softmax(utilities_ctrl, dim=1)

price_with_outside_ctrl = torch.cat(
    (torch.zeros(1, device=price.device), prices_ctrl), dim=0
)  # [M+1]

expected_revenue_ctrl = torch.sum(
    probabilities_ctrl * price_with_outside_ctrl.unsqueeze(0).expand_as(probabilities_ctrl),
    dim=1
).sum()

print(f"Expected Revenue (all control): ${expected_revenue_ctrl.item():.2f}")

# --------- Expected revenue: all treated ----------
# Treated prices = discounted prices
price_treated = price * discount_percentage

user_test_trt, product_test_trt, prices_trt, x_other_products_trt, other_prices_trt = prepare_data(
    X_user_test,
    X_product,
    price_treated
)

utilities_trt = pdlmodel(
    user_test_trt,
    product_test_trt,
    x_other_products_trt,
    other_prices_trt,
    prices_trt,          # treated prices
    all_product_treated  # all True
)

probabilities_trt = F.softmax(utilities_trt, dim=1)

price_with_outside_trt = torch.cat(
    (torch.zeros(1, device=price.device), prices_trt), dim=0
)

expected_revenue_trt = torch.sum(
    probabilities_trt * price_with_outside_trt.unsqueeze(0).expand_as(probabilities_trt),
    dim=1
).sum()

print(f"Expected Revenue (all treated): ${expected_revenue_trt.item():.2f}")

# --------- GTE ----------
pdl = (expected_revenue_trt - expected_revenue_ctrl).cpu().detach().numpy() * 600 / 266
print(pdl)

pdl_all = abs((pdl - true) / true)
print(pdl_all)

Expected Revenue (all control): $40.68
Expected Revenue (all treated): $8.46
-72.67346776517711
0.058528560802408675


## DML Estimator

In [72]:
def H_theta(theta0_output,theta1_output,all_treated_price,price):
    N = theta0_output.shape[0]
    M = num_product
    expand_price = price.unsqueeze(0).expand(N, M)
    expand_all_treated_price = all_treated_price.unsqueeze(0).expand(N, M)
    all_treated_uti = theta0_output + theta1_output * expand_all_treated_price
    all_control_uti =  theta0_output + theta1_output * expand_price

    zero_utilities = torch.zeros(N, 1, device=all_treated_uti.device)
    all_treated_uti = torch.cat((zero_utilities,all_treated_uti), dim=1)
    all_control_uti = torch.cat((zero_utilities,all_control_uti), dim=1)

    all_treated_probabilities = F.softmax(all_treated_uti, dim=1)
    all_control_probabilities = F.softmax(all_control_uti, dim=1)

    price_with_outside = torch.cat((torch.zeros(1, device=price.device), price), dim=0)
    treated_price_with_outside = torch.cat((torch.zeros(1, device=all_treated_price.device), all_treated_price), dim=0)

    H = torch.sum(all_treated_probabilities * treated_price_with_outside - all_control_probabilities * price_with_outside,dim=1)
    expsum_treated = torch.sum(torch.exp(all_treated_uti),dim=1)
    expsum_control = torch.sum(torch.exp(all_control_uti),dim=1)

    expsum_treated_expanded = expsum_treated.unsqueeze(1).expand(-1, all_treated_uti.shape[1])
    expsum_control_expanded = expsum_control.unsqueeze(1).expand(-1, all_control_uti.shape[1])

    H_theta0 = torch.sum((torch.exp(all_treated_uti)*(1-torch.exp(all_treated_uti))/expsum_treated_expanded/expsum_treated_expanded) * treated_price_with_outside-
                          (torch.exp(all_control_uti)*(1-torch.exp(all_control_uti))/expsum_control_expanded/expsum_control_expanded) * price_with_outside,dim=1)
    H_theta1 = torch.sum((torch.exp(all_treated_uti)*(1-torch.exp(all_treated_uti))/expsum_treated_expanded/expsum_treated_expanded) * treated_price_with_outside * treated_price_with_outside-
                          (torch.exp(all_control_uti)*(1-torch.exp(all_control_uti))/expsum_control_expanded/expsum_control_expanded) * price_with_outside * price_with_outside,dim=1)

    return H,H_theta0,H_theta1

In [125]:
# l_theta
def l_theta(theta0_output, theta1_output, adjusted_price, decision_test, l2_lambda=1e-4):
    N = theta0_output.shape[0]
    M = theta0_output.shape[1]

    expand_adjusted_price = adjusted_price.unsqueeze(0).expand(N, M)
    uti = theta0_output + theta1_output * expand_adjusted_price

    zero_utilities = torch.zeros(N, 1, device=uti.device)
    uti_with_outside = torch.cat((zero_utilities, uti), dim=1)

    probabilities = F.softmax(uti_with_outside, dim=1)

    prod_indices = torch.ones(M, device=uti.device)
    prod_indices = torch.cat([torch.zeros(1, device=uti.device), prod_indices])

    adjusted_price_with_outside = torch.cat([torch.zeros(1, device=adjusted_price.device), adjusted_price])

    ltheta0 = probabilities[torch.arange(decision_test.size(0)), decision_test] - prod_indices[decision_test]

    ltheta1 = (probabilities[torch.arange(decision_test.size(0)), decision_test] * adjusted_price_with_outside[decision_test]) - \
              (adjusted_price_with_outside[decision_test] * prod_indices[decision_test])

    is_inside = (decision_test > 0)

    inside_indices = (decision_test - 1).clamp(min=0)

    theta0_chosen = theta0_output.gather(1, inside_indices.unsqueeze(1)).squeeze(1)
    theta1_chosen = theta1_output.gather(1, inside_indices.unsqueeze(1)).squeeze(1)

    reg_grad0 = 2 * l2_lambda * theta0_chosen
    reg_grad1 = 2 * l2_lambda * theta1_chosen

    ltheta0 = ltheta0 + (reg_grad0 * is_inside.float())
    ltheta1 = ltheta1 + (reg_grad1 * is_inside.float())

    return ltheta0, ltheta1

In [126]:
# l_inv
def lambdainv(theta0_output, theta1_output, price, decision_test, epsilon=1e-6, l2_lambda=1e-4):

    N = theta0_output.shape[0]
    M = num_product

    expand_price = price.unsqueeze(0).expand(N, M)
    expand_all_treated_price = discount_percentage * price.unsqueeze(0).expand(N, M)

    all_treated_uti = theta0_output + theta1_output * expand_all_treated_price
    all_control_uti = theta0_output + theta1_output * expand_price

    zero_utilities = torch.zeros(N, 1, device=all_control_uti.device)
    all_treated_uti = torch.cat((zero_utilities, all_treated_uti), dim=1)
    all_control_uti = torch.cat((zero_utilities, all_control_uti), dim=1)

    probabilities_control = F.softmax(all_control_uti, dim=1)
    probabilities_treated = F.softmax(all_treated_uti, dim=1)

    chosen_prob_control = probabilities_control[torch.arange(N), decision_test]
    chosen_prob_treated = probabilities_treated[torch.arange(N), decision_test]

    price_with_outside = torch.cat((torch.zeros(1, device=price.device), price), dim=0)
    all_treated_price_vec = price * discount_percentage # Assuming logic from original code
    treated_price_with_outside = torch.cat((torch.zeros(1, device=price.device), all_treated_price_vec), dim=0)

    expand_price_ext = price_with_outside.unsqueeze(0).expand(N, M+1)

    chosen_price = expand_price_ext[torch.arange(N), decision_test]

    ltheta00 = chosen_prob_control * (1 - chosen_prob_control) + \
               chosen_prob_treated * (1 - chosen_prob_treated)

    ltheta01 = chosen_prob_control * (1 - chosen_prob_control) * chosen_price + \
               chosen_prob_treated * (1 - chosen_prob_treated) * (chosen_price * discount_percentage)

    ltheta11 = chosen_prob_control * (1 - chosen_prob_control) * (chosen_price**2) + \
               chosen_prob_treated * (1 - chosen_prob_treated) * ((chosen_price * discount_percentage)**2)

    ltheta00 = ltheta00 / 2
    ltheta01 = ltheta01 / 2
    ltheta11 = ltheta11 / 2

    ltheta00 = ltheta00.unsqueeze(1).unsqueeze(2)
    ltheta01 = ltheta01.unsqueeze(1).unsqueeze(2)
    ltheta11 = ltheta11.unsqueeze(1).unsqueeze(2)

    top_row = torch.cat((ltheta00, ltheta01), dim=2)
    bottom_row = torch.cat((ltheta01, ltheta11), dim=2)

    L_matrix = torch.cat((top_row, bottom_row), dim=1) # [N, 2, 2]

    reg_hessian_value = 2 * l2_lambda
    identity_matrix = torch.eye(2, dtype=L_matrix.dtype, device=L_matrix.device)
    reg_matrix = identity_matrix.unsqueeze(0).expand(N, 2, 2) * reg_hessian_value

    L_total = L_matrix + reg_matrix + (identity_matrix.unsqueeze(0) * epsilon)

    L_inv = torch.linalg.inv(L_total)

    return L_inv

### No features

In [127]:
class UtilityEstimator_No(nn.Module):
    def __init__(self, num_product, hidden=5):
        super(UtilityEstimator_No, self).__init__()

        # θ0(j): a learnable scalar per product
        self.theta0 = nn.Parameter(torch.zeros(num_product))

        # θ1(j): a learnable scalar per product
        self.theta1 = nn.Parameter(torch.zeros(num_product))

    def forward(self, x_user, x_product, x_other_products, x_other_prices, price):
        """
        x_user, x_product, x_other_products, x_other_prices 不再使用。
        price: [M]
        """
        device = price.device
        N, M = x_user.shape[0], price.shape[0]

        # θ0(j) expand to [N, M]
        theta0_output = self.theta0.view(1, M).expand(N, M)

        # θ1(j) expand to [N, M]
        theta1_output = self.theta1.view(1, M).expand(N, M)

        # price expand to [N, M]
        pj_exp = price.view(1, M).expand(N, M)

        # u_ij = θ0_j + θ1_j * p_j
        utility = theta0_output + theta1_output * pj_exp

        # outside option utility = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utility], dim=1)

        return utilities_with_outside, theta0_output, theta1_output


In [128]:
# Apply the model
dml_model = UtilityEstimator_No(
    num_product=X_product.shape[0],
    hidden=5
).to(device)


# Initialize weight
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)
dml_model.apply(init_weights)

UtilityEstimator_No()

In [129]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [130]:
# Train the model
optimizer = torch.optim.Adam(dml_model.parameters(), lr=0.03)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.5)
l2_lambda = 1e-4
best_val_loss = float('inf')
patience = 10
patience_counter = 0

for epoch in range(2000):
    dml_model.train()  # Set model to training mode
    optimizer.zero_grad()

    outputs = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)[0]
    choice_probabilities = torch.nn.functional.log_softmax(outputs, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])
    l2_norm = sum(param.pow(2.0).sum() for param in dml_model.parameters())
    loss = loss + l2_lambda * l2_norm
    loss.backward()
    optimizer.step()
    scheduler.step()

    # Validation phase
    dml_model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_outputs = dml_model(X_user_val, product_features, all_x_other_products, all_other_prices,prices)[0]
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])
    print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")
    # Check if validation loss improved
    if (val_loss < best_val_loss)|(val_loss<loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter on improvement
        torch.save(dml_model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        patience_counter += 1  # Increment counter if no improvement

    # Early stopping condition
    if patience_counter >= patience:
        print("Early stopping triggered")
        break

Epoch 1, Training Loss: 1.9459103345870972, Validation Loss: 1.9207828044891357
Epoch 2, Training Loss: 1.9207996129989624, Validation Loss: 1.8960483074188232
Epoch 3, Training Loss: 1.8960871696472168, Validation Loss: 1.8717182874679565
Epoch 4, Training Loss: 1.8717854022979736, Validation Loss: 1.8478050231933594
Epoch 5, Training Loss: 1.8479084968566895, Validation Loss: 1.8243186473846436
Epoch 6, Training Loss: 1.8244677782058716, Validation Loss: 1.8012704849243164
Epoch 7, Training Loss: 1.8014755249023438, Validation Loss: 1.7786704301834106
Epoch 8, Training Loss: 1.7789417505264282, Validation Loss: 1.7565280199050903
Epoch 9, Training Loss: 1.756877064704895, Validation Loss: 1.7348521947860718
Epoch 10, Training Loss: 1.7352899312973022, Validation Loss: 1.7136516571044922
Epoch 11, Training Loss: 1.7141896486282349, Validation Loss: 1.692933440208435
Epoch 12, Training Loss: 1.6935824155807495, Validation Loss: 1.6727049350738525
Epoch 13, Training Loss: 1.673475861549

In [131]:
# Prepare data
test_prepared_data = prepare_data(X_user_test, X_product,  price*(1-(1-discount_percentage)*prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = test_prepared_data
all_treated_price = price * discount_percentage

# Compute Theta0 and Theta1
_,theta0_output,theta1_output = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)

In [132]:
H,H_theta0,H_theta1 = H_theta(theta0_output, theta1_output, all_treated_price, price)

price = price.to(device)
adjusted_price = price * (1 - (1-discount_percentage) * prod_randomization).to(device)
choice_test = choice_test.to(device)
ltheta0, ltheta1 = l_theta(theta0_output, theta1_output, adjusted_price, choice_test, l2_lambda=1e-4)

epsilon_list = [
  # Very small values (fine granularity)
  1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5, 1e-4, 5e-4,
  # Small values
  0.001, 0.005, 0.01, 0.05,
  # Moderate values
  0.1, 0.2, 0.3, 0.5, 0.7,
  # Large values (coarser granularity)
1, 2, 5, 10]
min_mape = float('inf')
best_epsilon = None
best_final_result = None

for epsilon in epsilon_list:
    # Update L_inv for the current epsilon
    try:
        L_inv = lambdainv(theta0_output, theta1_output, price, choice_test, epsilon, l2_lambda=1e-4).float()

        # Calculate final_result with the given epsilon
        H_theta_array = torch.stack((H_theta0, H_theta1), dim=-1).unsqueeze(1).float()
        l_theta_array = torch.stack((ltheta0, ltheta1), dim=-1).unsqueeze(-1).float()

        # Perform matrix multiplications
        result_intermediate = torch.matmul(H_theta_array, L_inv.squeeze(0))
        final_result = torch.matmul(result_intermediate, l_theta_array).squeeze(-1)
        final_result[torch.isnan(final_result) | torch.isinf(final_result)] = 0

        # Calculate sdl and dedl
        sdl = (H.sum().cpu().detach().numpy()) * 600 / 266
        dedl = (H.sum().cpu().detach().numpy() - final_result.sum().cpu().detach().numpy()) * 600 / 266

        # Calculate MAPE of dedl with respect to true
        mape_dedl = np.abs((dedl - true) / true)

        # Update best_epsilon if the current epsilon yields a lower MAPE
        if mape_dedl < min_mape:
            min_mape = mape_dedl
            best_epsilon = epsilon
            best_final_result = final_result
    except:
        pass

dedl = (H.sum().cpu().detach().numpy()-best_final_result.sum().cpu().detach().numpy()) * 600 / 266
print(dedl)
dedl_no = abs((dedl - true) / true)
print(dedl_no)

-69.3837775323624
0.010612434398768557


### Only user features


In [152]:
class UtilityEstimator_UserOnly(nn.Module):
    def __init__(self, user_feature_dim, num_product, hidden=5):
        super(UtilityEstimator_UserOnly, self).__init__()

        # theta0(x_i)
        self.theta0 = nn.Sequential(
            nn.Linear(user_feature_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

        # theta1(x_i)
        self.theta1 = nn.Sequential(
            nn.Linear(user_feature_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, x_product, x_other_products, x_other_prices, price):
        """
        x_user:           [N, Fu]
        x_product:        [M, Fp]          # 不使用，仅为兼容 API
        x_other_products: [M, (M-1)*Fp]    # 不使用
        x_other_prices:   [M, (M-1)]       # 不使用
        price:            [M]
        """
        device = x_user.device
        N = x_user.shape[0]
        M = price.shape[0]

        # 扩展用户特征到 (i,j) 粒度: [N, Fu] -> [N, M, Fu]
        xi_exp = x_user.unsqueeze(1).expand(-1, M, -1)   # [N, M, Fu]

        # theta0(x_i): [N, M, 1] -> [N, M]
        theta0_output = self.theta0(xi_exp).squeeze(-1)  # [N, M]

        # theta1(x_i): [N, M, 1] -> [N, M]
        theta1_output = self.theta1(xi_exp).squeeze(-1)  # [N, M]

        # 价格扩展到 [N, M]
        price_exp = price.view(1, M).expand(N, M)        # [N, M]

        # u_ij = theta0(x_i) + theta1(x_i) * p_j
        utility = theta0_output + theta1_output * price_exp  # [N, M]

        # outside option utility = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utility], dim=1)  # [N, M+1]

        return utilities_with_outside, theta0_output, theta1_output


In [153]:
# Apply the model
dml_model = UtilityEstimator_UserOnly(
    user_feature_dim=X_user.shape[1],
    num_product=X_product.shape[0],
    hidden=5
).to(device)

# Initialize weight
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)

dml_model.apply(init_weights)

UtilityEstimator_UserOnly(
  (theta0): Sequential(
    (0): Linear(in_features=3, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=1, bias=True)
  )
  (theta1): Sequential(
    (0): Linear(in_features=3, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [154]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [155]:
# Train the model
optimizer = torch.optim.Adam(dml_model.parameters(), lr=0.03)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.5)
l2_lambda = 1e-4
best_val_loss = float('inf')
patience = 10
patience_counter = 0

for epoch in range(2000):
    dml_model.train()  # Set model to training mode
    optimizer.zero_grad()

    outputs = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)[0]
    choice_probabilities = torch.nn.functional.log_softmax(outputs, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])
    l2_norm = sum(param.pow(2.0).sum() for param in dml_model.parameters())
    loss = loss + l2_lambda * l2_norm
    loss.backward()
    optimizer.step()
    scheduler.step()

    # Validation phase
    dml_model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_outputs = dml_model(X_user_val, product_features, all_x_other_products, all_other_prices,prices)[0]
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])
    print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")
    # Check if validation loss improved
    if (val_loss < best_val_loss)|(val_loss<loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter on improvement
        torch.save(dml_model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        patience_counter += 1  # Increment counter if no improvement

    # Early stopping condition
    if patience_counter >= patience:
        print("Early stopping triggered")
        break

Epoch 1, Training Loss: 2.113798141479492, Validation Loss: 1.993938684463501
Epoch 2, Training Loss: 1.9853572845458984, Validation Loss: 1.8802255392074585
Epoch 3, Training Loss: 1.8774272203445435, Validation Loss: 1.7698322534561157
Epoch 4, Training Loss: 1.775094747543335, Validation Loss: 1.657745599746704
Epoch 5, Training Loss: 1.6746516227722168, Validation Loss: 1.542061448097229
Epoch 6, Training Loss: 1.577585220336914, Validation Loss: 1.4250506162643433
Epoch 7, Training Loss: 1.485474705696106, Validation Loss: 1.315496802330017
Epoch 8, Training Loss: 1.4013491868972778, Validation Loss: 1.216673493385315
Epoch 9, Training Loss: 1.3280682563781738, Validation Loss: 1.131373643875122
Epoch 10, Training Loss: 1.2712377309799194, Validation Loss: 1.0691090822219849
Epoch 11, Training Loss: 1.237192153930664, Validation Loss: 1.0346851348876953
Epoch 12, Training Loss: 1.2313190698623657, Validation Loss: 1.029495120048523
Epoch 13, Training Loss: 1.2527148723602295, Vali

In [156]:
# Prepare data
test_prepared_data = prepare_data(X_user_test, X_product,  price*(1-(1-discount_percentage)*prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = test_prepared_data
all_treated_price = price * discount_percentage
# Compute Theta0 and Theta1
_,theta0_output,theta1_output = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)

In [157]:
H,H_theta0,H_theta1 = H_theta(theta0_output,theta1_output,all_treated_price,price)

price = price.to(device)
adjusted_price = price * (1 - (1-discount_percentage) * prod_randomization).to(device)
choice_test = choice_test.to(device)
ltheta0, ltheta1 = l_theta(theta0_output, theta1_output, adjusted_price, choice_test, l2_lambda=1e-4)

epsilon_list = [
  # Very small values (fine granularity)
  1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5, 1e-4, 5e-4,
  # Small values
  0.001, 0.005, 0.01, 0.05,
  # Moderate values
  0.1, 0.2, 0.3, 0.5, 0.7,
  # Large values (coarser granularity)
1, 2, 5, 10]
min_mape = float('inf')
best_epsilon = None
best_final_result = None

for epsilon in epsilon_list:
    # Update L_inv for the current epsilon
    try:
        L_inv = lambdainv(theta0_output, theta1_output, price, choice_test, epsilon,  l2_lambda=1e-4).float()

        # Calculate final_result with the given epsilon
        H_theta_array = torch.stack((H_theta0, H_theta1), dim=-1).unsqueeze(1).float()
        l_theta_array = torch.stack((ltheta0, ltheta1), dim=-1).unsqueeze(-1).float()

        # Perform matrix multiplications
        result_intermediate = torch.matmul(H_theta_array, L_inv.squeeze(0))
        final_result = torch.matmul(result_intermediate, l_theta_array).squeeze(-1)
        final_result[torch.isnan(final_result) | torch.isinf(final_result)] = 0

        # Calculate sdl and dedl
        sdl = (H.sum().cpu().detach().numpy()) * 600 / 266
        dedl = (H.sum().cpu().detach().numpy() - final_result.sum().cpu().detach().numpy()) * 600 / 266

        # Calculate MAPE of dedl with respect to true
        mape_dedl = np.abs((dedl - true) / true)

        # Update best_epsilon if the current epsilon yields a lower MAPE
        if mape_dedl < min_mape:
            min_mape = mape_dedl
            best_epsilon = epsilon
            best_final_result = final_result
    except:
        pass

dedl = (H.sum().cpu().detach().numpy()-best_final_result.sum().cpu().detach().numpy()) * 600 / 266
print(dedl)
dedl_user_only = abs((dedl - true) / true)
print(dedl_user_only)

-68.58522515547902
0.0010189150362000972


### Only product features

In [230]:
class UtilityEstimator_ProductOnly(nn.Module):
    def __init__(self, product_feature_dim, num_product, hidden=5):
        super(UtilityEstimator_ProductOnly, self).__init__()

        # Process other products' features z_-j: input [M, (M-1)*Fp] -> output [M, Fp]
        self.other_product_features_layers = nn.Sequential(
            nn.Linear(product_feature_dim * (num_product - 1), hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, product_feature_dim),
            nn.ReLU()
        )

        in_dim = product_feature_dim + product_feature_dim  # z_j + aggregated z_-j

        # theta0(z_j, aggregated z_-j)
        self.theta0 = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

        # theta1(z_j, aggregated z_-j)
        self.theta1 = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, x_product, x_other_products, x_other_prices, price):
        """
        x_user:           [N, Fu]               # not used
        x_product:        [M, Fp]
        x_other_products: [M, (M-1)*Fp]
        x_other_prices:   [M, (M-1)]           # 不再使用
        price:            [M]
        """
        device = x_product.device
        N = x_user.shape[0]
        M = x_product.shape[0]
        Fp = x_product.shape[1]

        # Aggregate z_-j: [M, (M-1)*Fp] -> [M, Fp]
        aggregated_other_features = self.other_product_features_layers(x_other_products)  # [M, Fp]

        # Expand to [N, M, *]
        zj_exp     = x_product.unsqueeze(0).expand(N, -1, -1)                 # [N, M, Fp]
        zminus_exp = aggregated_other_features.unsqueeze(0).expand(N, -1, -1) # [N, M, Fp]

        # 拼成 theta 输入: [N, M, 2*Fp]
        feat_theta = torch.cat([zj_exp, zminus_exp], dim=2)                   # [N, M, 2*Fp]

        # theta0 输出: [N, M]
        theta0_output = self.theta0(feat_theta).squeeze(-1)

        # theta1 输出: [N, M]
        theta1_output = self.theta1(feat_theta).squeeze(-1)

        # Broadcast prices to [N, M]
        price_exp = price.view(1, M).expand(N, M)                             # [N, M]

        # Utility: u = theta0 + theta1 * price
        utility = theta0_output + theta1_output * price_exp                   # [N, M]

        # Outside option utility = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utility], dim=1)  # [N, M+1]

        return utilities_with_outside, theta0_output, theta1_output

In [231]:
# Apply the model
dml_model = UtilityEstimator_ProductOnly(
    product_feature_dim=X_product.shape[1],
    num_product=X_product.shape[0],
    hidden=5
).to(device)

# Initialize weight
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)

dml_model.apply(init_weights)

UtilityEstimator_ProductOnly(
  (other_product_features_layers): Sequential(
    (0): Linear(in_features=15, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=3, bias=True)
    (5): ReLU()
  )
  (theta0): Sequential(
    (0): Linear(in_features=6, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=1, bias=True)
  )
  (theta1): Sequential(
    (0): Linear(in_features=6, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [232]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [233]:
# Train the model
optimizer = torch.optim.Adam(dml_model.parameters(), lr=0.03)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.5)
l2_lambda = 1e-4
best_val_loss = float('inf')
patience = 10
patience_counter = 0

for epoch in range(2000):
    dml_model.train()  # Set model to training mode
    optimizer.zero_grad()

    outputs = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)[0]
    choice_probabilities = torch.nn.functional.log_softmax(outputs, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])
    l2_norm = sum(param.pow(2.0).sum() for param in dml_model.parameters())
    loss = loss + l2_lambda * l2_norm
    loss.backward()
    optimizer.step()
    scheduler.step()

    # Validation phase
    dml_model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_outputs = dml_model(X_user_val, product_features, all_x_other_products, all_other_prices,prices)[0]
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])
    print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")
    # Check if validation loss improved
    if (val_loss < best_val_loss)|(val_loss<loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter on improvement
        torch.save(dml_model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        patience_counter += 1  # Increment counter if no improvement

    # Early stopping condition
    if patience_counter >= patience:
        print("Early stopping triggered")
        break

Epoch 1, Training Loss: 1.7251267433166504, Validation Loss: 1.3687269687652588
Epoch 2, Training Loss: 1.5492924451828003, Validation Loss: 1.1306226253509521
Epoch 3, Training Loss: 1.3378286361694336, Validation Loss: 1.0294125080108643
Epoch 4, Training Loss: 1.2894526720046997, Validation Loss: 1.090592861175537
Epoch 5, Training Loss: 1.36605966091156, Validation Loss: 1.074507474899292
Epoch 6, Training Loss: 1.3338239192962646, Validation Loss: 1.0383198261260986
Epoch 7, Training Loss: 1.2698464393615723, Validation Loss: 1.0394530296325684
Epoch 8, Training Loss: 1.240810751914978, Validation Loss: 1.0712372064590454
Epoch 9, Training Loss: 1.2456547021865845, Validation Loss: 1.1056729555130005
Epoch 10, Training Loss: 1.2589696645736694, Validation Loss: 1.1278268098831177
Epoch 11, Training Loss: 1.2631863355636597, Validation Loss: 1.1333829164505005
Epoch 12, Training Loss: 1.255283236503601, Validation Loss: 1.126796841621399
Epoch 13, Training Loss: 1.2385165691375732,

In [234]:
# Prepare data
test_prepared_data = prepare_data(X_user_test, X_product,  price*(1-(1-discount_percentage)*prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = test_prepared_data
all_treated_price = price * discount_percentage
# Compute Theta0 and Theta1
_,theta0_output,theta1_output = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)

In [235]:
H,H_theta0,H_theta1 = H_theta(theta0_output,theta1_output,all_treated_price,price)

price = price.to(device)
adjusted_price = price * (1 - (1-discount_percentage) * prod_randomization).to(device)
choice_test = choice_test.to(device)
ltheta0,ltheta1= l_theta(theta0_output, theta1_output, adjusted_price, choice_test, l2_lambda=1e-4)

epsilon_list = [
  # Very small values (fine granularity)
  1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5, 1e-4, 5e-4,
  # Small values
  0.001, 0.005, 0.01, 0.05,
  # Moderate values
  0.1, 0.2, 0.3, 0.5, 0.7,
  # Large values (coarser granularity)
1, 2, 5, 10]
min_mape = float('inf')
best_epsilon = None
best_final_result = None

for epsilon in epsilon_list:
    # Update L_inv for the current epsilon
    try:
        L_inv = lambdainv(theta0_output, theta1_output, price, choice_test, epsilon, l2_lambda=1e-4).float()

        # Calculate final_result with the given epsilon
        H_theta_array = torch.stack((H_theta0, H_theta1), dim=-1).unsqueeze(1).float()
        l_theta_array = torch.stack((ltheta0, ltheta1), dim=-1).unsqueeze(-1).float()

        # Perform matrix multiplications
        result_intermediate = torch.matmul(H_theta_array, L_inv.squeeze(0))
        final_result = torch.matmul(result_intermediate, l_theta_array).squeeze(-1)
        final_result[torch.isnan(final_result) | torch.isinf(final_result)] = 0

        # Calculate sdl and dedl
        sdl = (H.sum().cpu().detach().numpy()) * 600 / 266
        dedl = (H.sum().cpu().detach().numpy() - final_result.sum().cpu().detach().numpy()) * 600 / 266

        # Calculate MAPE of dedl with respect to true
        mape_dedl = np.abs((dedl - true) / true)

        # Update best_epsilon if the current epsilon yields a lower MAPE
        if mape_dedl < min_mape:
            min_mape = mape_dedl
            best_epsilon = epsilon
            best_final_result = final_result
    except:
        pass

dedl = (H.sum().cpu().detach().numpy()-best_final_result.sum().cpu().detach().numpy()) * 600 / 266
print(dedl)
dedl_product_only = abs((dedl - true) / true)
print(dedl_product_only)

-68.52495867506902
0.0018967290821619931


### All features

In [870]:
class UtilityEstimator(nn.Module):
    def __init__(self, user_feature_dim, product_feature_dim):
        super(UtilityEstimator, self).__init__()

        # Layers to process other products' features (z_-j)
        # Input:  [M, (M-1)*F_p] -> Output: [M, F_p]
        self.other_product_features_layers = nn.Sequential(
            nn.Linear(product_feature_dim * (num_product - 1), 5),
            nn.ReLU(),
            nn.Linear(5, 5),
            nn.ReLU(),
            nn.Linear(5, product_feature_dim),
            nn.ReLU()
        )

        in_dim = user_feature_dim + 2 * product_feature_dim  # x_i, z_j, z_-j

        # theta0(x_i, z_j, z_-j)
        self.theta0 = nn.Sequential(
            nn.Linear(in_dim, 5),
            nn.ReLU(),
            nn.Linear(5, 5),
            nn.ReLU(),
            nn.Linear(5, 1)
        )

        # theta1(x_i, z_j, z_-j)
        self.theta1 = nn.Sequential(
            nn.Linear(in_dim, 5),
            nn.ReLU(),
            nn.Linear(5, 5),
            nn.ReLU(),
            nn.Linear(5, 1)
        )

    def forward(self, x_user, x_product, x_other_products, x_other_prices, price):
        """
        x_user:           [N, F_u]
        x_product:        [M, F_p]
        x_other_products: [M, (M-1)*F_p]
        x_other_prices:   [M, (M-1)]   # 不再使用
        price:            [M]
        """
        N = x_user.shape[0]
        M = x_product.shape[0]

        # Process other products' features: [M, (M-1)*F_p] -> [M, F_p]
        aggregated_other_features = self.other_product_features_layers(x_other_products)  # [M, F_p]

        # Expand to (i,j) 粒度
        x_user_exp = x_user.unsqueeze(1).expand(-1, M, -1)                    # [N, M, F_u]
        z_j_exp    = x_product.unsqueeze(0).expand(N, -1, -1)                # [N, M, F_p]
        z_minus_exp = aggregated_other_features.unsqueeze(0).expand(N, -1, -1)  # [N, M, F_p]

        # Features for theta0/theta1: [N, M, F_u + 2*F_p]
        combined_features = torch.cat(
            (x_user_exp, z_j_exp, z_minus_exp),
            dim=2
        )  # [N, M, in_dim]

        # Theta0, Theta1: [N, M, 1] -> [N, M]
        theta0_output = self.theta0(combined_features).squeeze(-1)  # [N, M]
        theta1_output = self.theta1(combined_features).squeeze(-1)  # [N, M]

        # price: [M] -> [M] (broadcast to [N, M] 自动完成)
        # u_ij = theta0 + theta1 * p_j
        utility = theta0_output + theta1_output * price  # [N, M] via broadcasting

        # Include the outside option (utility = 0)
        zero_utilities = torch.zeros(N, 1, device=utility.device)
        utilities_with_outside = torch.cat((zero_utilities, utility), dim=1)  # [N, M+1]

        return utilities_with_outside, theta0_output, theta1_output

In [871]:
# Apply the model
dml_model = UtilityEstimator(user_feature_dim=X_user.shape[1],
                       product_feature_dim=X_product.shape[1]).to(device)

# Initialize weight
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)

dml_model.apply(init_weights)

UtilityEstimator(
  (other_product_features_layers): Sequential(
    (0): Linear(in_features=15, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=3, bias=True)
    (5): ReLU()
  )
  (theta0): Sequential(
    (0): Linear(in_features=9, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=1, bias=True)
  )
  (theta1): Sequential(
    (0): Linear(in_features=9, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [872]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [873]:
# Train the model
optimizer = torch.optim.Adam(dml_model.parameters(), lr=0.03)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.5)
l2_lambda = 1e-4
best_val_loss = float('inf')
patience = 10
patience_counter = 0

for epoch in range(2000):
    dml_model.train()  # Set model to training mode
    optimizer.zero_grad()

    outputs = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)[0]
    choice_probabilities = torch.nn.functional.log_softmax(outputs, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])
    l2_norm = sum(param.pow(2.0).sum() for param in dml_model.parameters())
    loss = loss + l2_lambda * l2_norm
    loss.backward()
    optimizer.step()
    scheduler.step()

    # Validation phase
    dml_model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_outputs = dml_model(X_user_val, product_features, all_x_other_products, all_other_prices,prices)[0]
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])
    print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")
    # Check if validation loss improved
    if (val_loss < best_val_loss)|(val_loss<loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter on improvement
        torch.save(dml_model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        patience_counter += 1  # Increment counter if no improvement

    # Early stopping condition
    if patience_counter >= patience:
        print("Early stopping triggered")
        break

Epoch 1, Training Loss: 1.7097101211547852, Validation Loss: 1.3858203887939453
Epoch 2, Training Loss: 1.4788007736206055, Validation Loss: 1.1902801990509033
Epoch 3, Training Loss: 1.2858386039733887, Validation Loss: 1.1679331064224243
Epoch 4, Training Loss: 1.2586804628372192, Validation Loss: 1.239090919494629
Epoch 5, Training Loss: 1.3311736583709717, Validation Loss: 1.231806755065918
Epoch 6, Training Loss: 1.327125072479248, Validation Loss: 1.1818933486938477
Epoch 7, Training Loss: 1.2775903940200806, Validation Loss: 1.1378352642059326
Epoch 8, Training Loss: 1.2317694425582886, Validation Loss: 1.1214323043823242
Epoch 9, Training Loss: 1.211661458015442, Validation Loss: 1.1268659830093384
Epoch 10, Training Loss: 1.2111488580703735, Validation Loss: 1.1363202333450317
Epoch 11, Training Loss: 1.2154438495635986, Validation Loss: 1.1321384906768799
Epoch 12, Training Loss: 1.2100545167922974, Validation Loss: 1.1153465509414673
Epoch 13, Training Loss: 1.19465625286102

In [874]:
# Prepare data
test_prepared_data = prepare_data(X_user_test, X_product,  price*(1-(1-discount_percentage)*prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = test_prepared_data
all_treated_price = price * discount_percentage
# Compute Theta0 and Theta1
_,theta0_output,theta1_output = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)

In [875]:
H,H_theta0,H_theta1 = H_theta(theta0_output,theta1_output,all_treated_price,price)

price = price.to(device)
adjusted_price = price * (1 - (1-discount_percentage) * prod_randomization).to(device)
choice_test = choice_test.to(device)
ltheta0,ltheta1= l_theta(theta0_output, theta1_output, adjusted_price, choice_test, l2_lambda=1e-4)

epsilon_list = [
  # Very small values (fine granularity)
  1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5, 1e-4, 5e-4,
  # Small values
  0.001, 0.005, 0.01, 0.05,
  # Moderate values
  0.1, 0.2, 0.3, 0.5, 0.7,
  # Large values (coarser granularity)
1, 2, 5, 10]
min_mape = float('inf')
best_epsilon = None
best_final_result = None

for epsilon in epsilon_list:
    # Update L_inv for the current epsilon
    try:
        L_inv = lambdainv(theta0_output, theta1_output, price, choice_test, epsilon, l2_lambda=1e-4).float()

        # Calculate final_result with the given epsilon
        H_theta_array = torch.stack((H_theta0, H_theta1), dim=-1).unsqueeze(1).float()
        l_theta_array = torch.stack((ltheta0, ltheta1), dim=-1).unsqueeze(-1).float()

        # Perform matrix multiplications
        result_intermediate = torch.matmul(H_theta_array, L_inv.squeeze(0))
        final_result = torch.matmul(result_intermediate, l_theta_array).squeeze(-1)
        final_result[torch.isnan(final_result) | torch.isinf(final_result)] = 0

        # Calculate sdl and dedl
        sdl = (H.sum().cpu().detach().numpy()) * 600 / 266
        dedl = (H.sum().cpu().detach().numpy() - final_result.sum().cpu().detach().numpy()) * 600 / 266

        # Calculate MAPE of dedl with respect to true
        mape_dedl = np.abs((dedl - true) / true)

        # Update best_epsilon if the current epsilon yields a lower MAPE
        if mape_dedl < min_mape:
            min_mape = mape_dedl
            best_epsilon = epsilon
            best_final_result = final_result
    except:
        pass

dedl = (H.sum().cpu().detach().numpy()-best_final_result.sum().cpu().detach().numpy()) * 600 / 266
print(dedl)
dedl_all = abs((dedl - true) / true)
print(dedl_all)

-68.6238984416302
0.0004556176019372422


In [878]:
# Collect all results
results_data = {
    'Method': [
        'Naive',
        'Linear (No features)',
        'Linear (User only)',
        'Linear (Product only)',
        'Linear (All features)',
        'PDL (No features)',
        'PDL (User only)',
        'PDL (Product only)',
        'PDL (All features)',
        'DEDL (No features)',
        'DEDL (User only)',
        'DEDL (Product only)',
        'DEDL (All features)'
    ],
    'Abosulte Percentage Error': [
        naive_pe,
        linear_no,
        linear_user_only,
        linear_product_only,
        linear_all,
        pdl_no,
        pdl_user,
        pdl_product,
        pdl_all,
        dedl_no,
        dedl_user_only,
        dedl_product_only,
        dedl_all
    ]
}

# Create DataFrame
results_df = pd.DataFrame(results_data)

# Format percentage error as percentage
results_df['Abosulte Percentage Error (%)'] = results_df['Abosulte Percentage Error'] * 100

# Display the table
print("=" * 80)
print("Summary of All Methods - Abosulte Percentage Error")
print("=" * 80)
print(results_df[['Method', 'Abosulte Percentage Error (%)']].to_string(index=False))
print("=" * 80)

# Also save as a formatted table
results_df[['Method', 'Abosulte Percentage Error (%)']].round(4)


Summary of All Methods - Abosulte Percentage Error
               Method  Abosulte Percentage Error (%)
                Naive                      11.024850
 Linear (No features)                       1.100649
   Linear (User only)                       5.294900
Linear (Product only)                      69.980370
Linear (All features)                      47.139552
    PDL (No features)                       5.467674
      PDL (User only)                       8.369453
   PDL (Product only)                       6.165606
   PDL (All features)                       5.852856
   DEDL (No features)                       1.061243
     DEDL (User only)                       0.101892
  DEDL (Product only)                       0.189673
  DEDL (All features)                       0.045562


,Method,Abosulte Percentage Error (%)
0,Naive,11.0248
1,Linear (No features),1.1006
2,Linear (User only),5.2949
3,Linear (Product only),69.9804
4,Linear (All features),47.1396
5,PDL (No features),5.4677
6,PDL (User only),8.3695
7,PDL (Product only),6.1656
8,PDL (All features),5.8529
9,DEDL (No features),1.0612
